In [1]:
import pandas as pd
silver_df_main = pd.read_csv('data/sentence_level_evidence.csv')
silver_df_main

,SUBJECT_ID,HADM_ID,TEXT,ICD9_CODE
0,55677,108847.0,Admission Date: [**2134-4-5**] D...,"[""[('This 79 year old male who is known to car..."
1,55122,160937.0,Admission Date: [**2192-3-27**] ...,['[(\'Patient underwent uncomplicated penile i...
2,11892,193055.0,Admission Date: [**2152-6-26**] Dischar...,"[""[('Mr. [**Known lastname 36243**] is a 67-ye..."
3,13913,153057.0,Admission Date: [**2152-3-9**] Discharg...,"[""[\n ('6. Diabetes mellitus.', '250.00'),..."
4,21553,188632.0,Admission Date: [**2184-12-24**] ...,"[""[('Patient recorded as having No Known Aller..."
...,...,...,...,...
1445,70182,182356.0,Admission Date: [**2110-2-9**] D...,"[""[('75 year old female with history of mitral..."
1446,25235,182008.0,Admission Date: [**2167-6-10**] ...,"['[\n (""52 yo M with PMH DM I, CAD s/p CABG..."
1447,18736,152789.0,Admission Date: [**2109-8-30**] ...,"[""[\n ('Patient recorded as having No Known..."
1448,42757,167098.0,Admission Date: [**2154-10-6**] ...,"[""[\n ('The patient is a 59-year-old man wi..."


In [2]:
print(eval(silver_df_main.iloc[0,-1])), print(type(eval(silver_df_main.iloc[0,-1])))

["[('This 79 year old male who is known to cardiac surgery with critical aortic stenosis, having refused surgical intervention in the past, s/p valuloplasy x2 [**2132-10-15**]/[**2133-12-15**], who was transferred from an OSH for acute chest pain with troponin bump to 4.4.', '427.31'), ('He was transfered to [**Hospital1 18**] for further mangangment of aortic stenosis.', '427.31'), ('IMPRESSION: Severe aortic valve stenosis.', '427.31')]", "[('Novolog insulin 8 units [**Hospital1 **] (before breakfast and before\\nsupper)', '250.02'),\n('Insulin lispro', '250.02'),\n('metoprolol tartrate 25 mg Tablet Sig: One (1) Tablet PO BID\\n(2 times a day)', '272.0'),\n('Lasix 80 mg Tablet Sig: One (1) Tablet PO twice a day for 2\\nweeks.', '599.70'),\n('Lasix 40mg QDAY', '599.70'),\n('magnesium hydroxide 400 mg/5 mL Suspension Sig: Thirty (30)\\nML PO HS (at bedtime) as needed for constipation.', '682.9'),\n('potassium chloride 20 mEq Tablet, ER Particles/Crystals Sig:\\nOne (1) Tablet, ER Parti

(None, None)

In [3]:
# import re
# icd_regex = r'(?:\b\w+\b\s*)?\b\d{3,5}\.?\d{2,4}\b'
# code_list = []
# seen_codes = set()
# for x in eval(silver_df_main.iloc[0,-1]):
#     # code_list.append(re.findall(icd_regex, x))
#     codes = re.findall(icd_regex, x)
#     for code in codes:
#         if code not in seen_codes:
#             seen_codes.add(code)
#             code_list.append(code)
#     # print(x)

# print(code_list)

## Processing the data

In [4]:
candidate_list = """
    V70.0 - Routine Medical Exam
    V76.2 - Screening for Malignant Neoplasm of Cervix
    401.1 - Benign Hypertension
    V76.44 - Screening for Malignant Neoplasm of Prostate
    250.00 - Diabetes Mellitus without Complications, Type II or Unspecified Type
    401.9 - Hypertension, Unspecified
    780.79 - Other Malaise and Fatigue
    599.0 - Urinary Tract Infection, Site Not Specified
    272.4 - Other and Unspecified Hyperlipidemia
    V72.60 - Laboratory Examination, Unspecified
    244.9 - Hypothyroidism, Unspecified
    V58.69 - Long-term (current) Use of Other Medications
    788.1 - Dysuria
    272.2 - Mixed Hyperlipidemia
    272.0 - Pure Hypercholesterolemia
    268.9 - Unspecified Vitamin D Deficiency
    462 - Acute Pharyngitis
    250.02 - Diabetes Mellitus without Complications, Type II or Unspecified Type, Uncontrolled
    V58.61 - Long-term (current) Use of Anticoagulants
    789.00 - Abdominal Pain, Unspecified Site
    285.9 - Anemia, Unspecified
    427.31 - Atrial Fibrillation
    599.70 - Hematuria, Unspecified
    787.91 - Diarrhea
    780.60 - Fever, Unspecified
    V69.2 - High Risk Sexual Behavior
    V72.62 - Laboratory Examination, General Medical Examination
    790.6 - Abnormal Blood Chemistry
    595.0 - Acute Cystitis
    257.2 - Testicular Hypofunction
    244.8 - Acquired Hypothyroidism
    616.10 - Vaginitis and Vulvitis, Unspecified
    V20.2 - Routine Child Health Examination
    786.2 - Cough
    790.29 - Other Abnormal Glucose
    530.81 - Esophageal Reflux
    V73.88 - Screening for Chlamydial Disease
    782.3 - Edema
    V77.91 - Screening for Lipoid Disorders
    780.4 - Dizziness and Giddiness
    V72.31 - Routine Gynecological Examination
    V22.1 - Supervision of Other Normal Pregnancy
    280.9 - Iron Deficiency Anemia, Unspecified
    300.00 - Anxiety State, Unspecified
    682.9 - Cellulitis and Abscess, Unspecified
    789.07 - Abdominal Pain, Generalized
    719.40 - Pain in Joint, Unspecified
    788.41 - Urinary Frequency
    784.0 - Headache
    783.1 - Abnormal Weight Gain
"""
import re

regex_code_desc = re.compile(r'(\b[V\d.]+)\s+-\s+(.*)')

code_descs = regex_code_desc.findall(candidate_list)


In [5]:
code_descs_dict = {code:desc for code, desc in code_descs}
code_descs_dict

{'V70.0': 'Routine Medical Exam',
 'V76.2': 'Screening for Malignant Neoplasm of Cervix',
 '401.1': 'Benign Hypertension',
 'V76.44': 'Screening for Malignant Neoplasm of Prostate',
 '250.00': 'Diabetes Mellitus without Complications, Type II or Unspecified Type',
 '401.9': 'Hypertension, Unspecified',
 '780.79': 'Other Malaise and Fatigue',
 '599.0': 'Urinary Tract Infection, Site Not Specified',
 '272.4': 'Other and Unspecified Hyperlipidemia',
 'V72.60': 'Laboratory Examination, Unspecified',
 '244.9': 'Hypothyroidism, Unspecified',
 'V58.69': 'Long-term (current) Use of Other Medications',
 '788.1': 'Dysuria',
 '272.2': 'Mixed Hyperlipidemia',
 '272.0': 'Pure Hypercholesterolemia',
 '268.9': 'Unspecified Vitamin D Deficiency',
 '462': 'Acute Pharyngitis',
 '250.02': 'Diabetes Mellitus without Complications, Type II or Unspecified Type, Uncontrolled',
 'V58.61': 'Long-term (current) Use of Anticoagulants',
 '789.00': 'Abdominal Pain, Unspecified Site',
 '285.9': 'Anemia, Unspecified

In [6]:
silver_df_main_og = silver_df_main

In [7]:
import ast

sentence_list = []
for x in eval(silver_df_main_og.iloc[0, -1]):
    datas = ast.literal_eval(x)
    for data in datas:
        sentence_list.append(data)

for i, snt in enumerate(sentence_list):
    print(f'{i+1}.) ',snt[1])

1.)  427.31
2.)  427.31
3.)  427.31
4.)  250.02
5.)  250.02
6.)  272.0
7.)  599.70
8.)  599.70
9.)  682.9
10.)  287.9
11.)  250.00
12.)  250.02
13.)  427.31
14.)  427.31


In [8]:
# import re
# icd_regex = r'(?:\b\w+\b\s*)?\b\d{3,5}\.?\d{2,4}\b'
# code_list = []
# seen_codes = set()
# for x in eval(silver_df_main.iloc[0,-1]):
#     # code_list.append(re.findall(icd_regex, x))
#     codes = re.findall(icd_regex, x)
#     for code in codes:
#         if code not in seen_codes:
#             seen_codes.add(code)
#             code_list.append(code)
#     # print(x)

# print(code_list)


In [9]:
doc_list = silver_df_main['ICD9_CODE'].tolist()

In [10]:
from tqdm import tqdm
doc_sentences = []
doc_no = 0
doc_no_not_proccessed = []
for doc in tqdm(doc_list):
    try:
        sentence_list = []
        for x in eval(doc):
            datas = ast.literal_eval(x)
            for data in datas:
                sentence_list.append(data)
        
        if len(sentence_list) < 5: # If the sentence level evidences contain less than 3 sentences we will remove that document
            raise Exception("Too less sentences")
        doc_sentences.append(sentence_list)
        doc_no +=1
    except:
        doc_no_not_proccessed.append(doc_no)
        doc_no +=1

100%|██████████| 1450/1450 [00:00<00:00, 16632.41it/s]


In [11]:
len(doc_no_not_proccessed) # This documents have some sort of errors and have to be removed 

619

**Now we need to determine the maximum number of sentences we can keep for each document, for the sake of simplicity I'll take it to be the minimum number of evidence sentences in each Document**

In [12]:
len(doc_sentences[0])

14

In [13]:
min_sentences = 2**31

for no_sent in doc_sentences:
    if min_sentences > len(no_sent):
        min_sentences = len(no_sent)

print(min_sentences)


5


The Minimum number of sentences in a document is 3 so for all documents we'll keep the 1st 3 sentences only

In [14]:

xD = []
for index in silver_df_main.index:
    try:
        if index in doc_no_not_proccessed:
            silver_df_main = silver_df_main.drop(index, axis=0)
    except:
        xD.append(index)
        # print(f"Index that couldn't be dropped: {index}")
len(xD)

0

In [15]:
silver_df_main = silver_df_main.reset_index().drop('index', axis = 1)

In [16]:
silver_df_main

,SUBJECT_ID,HADM_ID,TEXT,ICD9_CODE
0,55677,108847.0,Admission Date: [**2134-4-5**] D...,"[""[('This 79 year old male who is known to car..."
1,55122,160937.0,Admission Date: [**2192-3-27**] ...,['[(\'Patient underwent uncomplicated penile i...
2,88157,168232.0,Admission Date: [**2190-2-23**] ...,"['[(""Patient is a 51 yo female with no cardiac..."
3,31290,159495.0,Admission Date: [**2201-10-14**] ...,"[""[\n('Patient is 58-year-old man with history..."
4,17341,151110.0,Admission Date: [**2132-6-9**] D...,"[""[\n ('The patient remained on the surgical ..."
...,...,...,...,...
826,93628,134497.0,Admission Date: [**2177-3-24**] ...,"[""[('Social History: Patient had been maried f..."
827,14667,172924.0,Admission Date: [**2138-8-7**] D...,"[""[('Pt has >100pk/yrs and continues to smoke'..."
828,25235,182008.0,Admission Date: [**2167-6-10**] ...,"['[\n (""52 yo M with PMH DM I, CAD s/p CABG..."
829,18736,152789.0,Admission Date: [**2109-8-30**] ...,"[""[\n ('Patient recorded as having No Known..."


In [17]:
len(doc_sentences)

831

In [18]:
silver_df_main['doc_sentences'] = pd.Series([x[:5] for x in doc_sentences], index = silver_df_main.index)

In [19]:
# silver_df_main['doc_sentences'] = silver_df_main['doc_sentences'][:5]

In [20]:
for x in silver_df_main['doc_sentences'].to_list():
    if len(x) != 5:
        raise Exception('Some sentences are not equal to 5')
print("We're Good to proceed")
    

We're Good to proceed


In [21]:
silver_df_main

,SUBJECT_ID,HADM_ID,TEXT,ICD9_CODE,doc_sentences
0,55677,108847.0,Admission Date: [**2134-4-5**] D...,"[""[('This 79 year old male who is known to car...",[(This 79 year old male who is known to cardia...
1,55122,160937.0,Admission Date: [**2192-3-27**] ...,['[(\'Patient underwent uncomplicated penile i...,[(Patient underwent uncomplicated penile impla...
2,88157,168232.0,Admission Date: [**2190-2-23**] ...,"['[(""Patient is a 51 yo female with no cardiac...",[(Patient is a 51 yo female with no cardiac hi...
3,31290,159495.0,Admission Date: [**2201-10-14**] ...,"[""[\n('Patient is 58-year-old man with history...",[(Patient is 58-year-old man with history of D...
4,17341,151110.0,Admission Date: [**2132-6-9**] D...,"[""[\n ('The patient remained on the surgical ...",[(The patient remained on the surgical service...
...,...,...,...,...,...
826,93628,134497.0,Admission Date: [**2177-3-24**] ...,"[""[('Social History: Patient had been maried f...",[(Social History: Patient had been maried for ...
827,14667,172924.0,Admission Date: [**2138-8-7**] D...,"[""[('Pt has >100pk/yrs and continues to smoke'...","[(Pt has >100pk/yrs and continues to smoke, 25..."
828,25235,182008.0,Admission Date: [**2167-6-10**] ...,"['[\n (""52 yo M with PMH DM I, CAD s/p CABG...","[(52 yo M with PMH DM I, CAD s/p CABG x2, asth..."
829,18736,152789.0,Admission Date: [**2109-8-30**] ...,"[""[\n ('Patient recorded as having No Known...",[(Patient recorded as having No Known Allergie...


### Load the Gold levels (To reduce complexity of processing we'll use the ICD Codes predicted by GPT-4)

In [22]:
gold_df = pd.read_csv('data/silver_labels.csv')


In [23]:
gold_df = gold_df.iloc[:1450, :]
gold_df

,SUBJECT_ID,HADM_ID,TEXT,ICD9_CODE,encoded_ICD9_CODES
0,55677,108847.0,Admission Date: [**2134-4-5**] D...,"['427.31', 'V58.61', '401.9', '272.0', '272.2'...",[0 0 0 ... 0 0 0]
1,55122,160937.0,Admission Date: [**2192-3-27**] ...,"['996.1', '996.42', '414.00', 'V45.81', 'V58.6...",[0 0 0 ... 0 0 0]
2,11892,193055.0,Admission Date: [**2152-6-26**] Dischar...,"['401.9', '250.00', '250.02', '427.31', '427.3...",[0 0 0 ... 0 0 0]
3,13913,153057.0,Admission Date: [**2152-3-9**] Discharg...,"['578.9', '414.00', '745.5', '428.0', '250.00'...",[0 0 0 ... 0 0 0]
4,21553,188632.0,Admission Date: [**2184-12-24**] ...,"['276.7', '276.1', '458.0', 'E878.2', 'E878.7']",[0 0 0 ... 0 0 0]
...,...,...,...,...,...
1445,70182,182356.0,Admission Date: [**2110-2-9**] D...,"['426.4', '394.1', '396.3', '429.3', '38.85', ...",[0 0 0 ... 0 0 0]
1446,25235,182008.0,Admission Date: [**2167-6-10**] ...,"['190.0', '198.5', '416.0', '491.21', '518.4',...",[0 0 0 ... 0 0 0]
1447,18736,152789.0,Admission Date: [**2109-8-30**] ...,"['427.31', '599.0', '276.9', '518.82', '428.0'...",[0 0 0 ... 0 0 0]
1448,42757,167098.0,Admission Date: [**2154-10-6**] ...,"['349.82', '276.2', '458.29', '038.9', '427.31...",[0 0 0 ... 0 0 0]


In [24]:
for index in gold_df.index:
    if index in doc_no_not_proccessed:
        gold_df = gold_df.drop(index, axis=0)

gold_df = gold_df.reset_index().drop('index', axis=1)
gold_df

,SUBJECT_ID,HADM_ID,TEXT,ICD9_CODE,encoded_ICD9_CODES
0,55677,108847.0,Admission Date: [**2134-4-5**] D...,"['427.31', 'V58.61', '401.9', '272.0', '272.2'...",[0 0 0 ... 0 0 0]
1,55122,160937.0,Admission Date: [**2192-3-27**] ...,"['996.1', '996.42', '414.00', 'V45.81', 'V58.6...",[0 0 0 ... 0 0 0]
2,88157,168232.0,Admission Date: [**2190-2-23**] ...,"['424.0', '396.3', '493.90', '491.21', '424.0'...",[0 0 0 ... 0 0 0]
3,31290,159495.0,Admission Date: [**2201-10-14**] ...,"['427.31', '250.02', '780.60', 'V58.61', '599....",[0 0 0 ... 0 0 0]
4,17341,151110.0,Admission Date: [**2132-6-9**] D...,"['01.04', '32.0', '41.2', '34.92', '38.09', '8...",[0 0 0 ... 0 0 0]
...,...,...,...,...,...
826,93628,134497.0,Admission Date: [**2177-3-24**] ...,"['800.00', '850.9', '305.00', '303.90', 'V87.3...",[0 0 0 ... 0 0 0]
827,14667,172924.0,Admission Date: [**2138-8-7**] D...,"['491.21', '250.00', '244.9', '305.1', '276.8'...",[0 0 0 ... 0 0 0]
828,25235,182008.0,Admission Date: [**2167-6-10**] ...,"['190.0', '198.5', '416.0', '491.21', '518.4',...",[0 0 0 ... 0 0 0]
829,18736,152789.0,Admission Date: [**2109-8-30**] ...,"['427.31', '599.0', '276.9', '518.82', '428.0'...",[0 0 0 ... 0 0 0]


In [25]:
gold_df['ICD9_CODE'] = pd.Series([eval(x) for x in gold_df['ICD9_CODE']], index=gold_df.index)

In [26]:
gold_df

,SUBJECT_ID,HADM_ID,TEXT,ICD9_CODE,encoded_ICD9_CODES
0,55677,108847.0,Admission Date: [**2134-4-5**] D...,"[427.31, V58.61, 401.9, 272.0, 272.2, 244.9, V...",[0 0 0 ... 0 0 0]
1,55122,160937.0,Admission Date: [**2192-3-27**] ...,"[996.1, 996.42, 414.00, V45.81, V58.61, 443.9,...",[0 0 0 ... 0 0 0]
2,88157,168232.0,Admission Date: [**2190-2-23**] ...,"[424.0, 396.3, 493.90, 491.21, 424.0, 795.00, ...",[0 0 0 ... 0 0 0]
3,31290,159495.0,Admission Date: [**2201-10-14**] ...,"[427.31, 250.02, 780.60, V58.61, 599.0, 682.9,...",[0 0 0 ... 0 0 0]
4,17341,151110.0,Admission Date: [**2132-6-9**] D...,"[01.04, 32.0, 41.2, 34.92, 38.09, 87.66, 22.56...",[0 0 0 ... 0 0 0]
...,...,...,...,...,...
826,93628,134497.0,Admission Date: [**2177-3-24**] ...,"[800.00, 850.9, 305.00, 303.90, V87.39, 348.30...",[0 0 0 ... 0 0 0]
827,14667,172924.0,Admission Date: [**2138-8-7**] D...,"[491.21, 250.00, 244.9, 305.1, 276.8, 293.0, 2...",[0 0 0 ... 0 0 0]
828,25235,182008.0,Admission Date: [**2167-6-10**] ...,"[190.0, 198.5, 416.0, 491.21, 518.4, 785.52, V...",[0 0 0 ... 0 0 0]
829,18736,152789.0,Admission Date: [**2109-8-30**] ...,"[427.31, 599.0, 276.9, 518.82, 428.0, 161.9, 2...",[0 0 0 ... 0 0 0]


In [27]:
silver_df_main

,SUBJECT_ID,HADM_ID,TEXT,ICD9_CODE,doc_sentences
0,55677,108847.0,Admission Date: [**2134-4-5**] D...,"[""[('This 79 year old male who is known to car...",[(This 79 year old male who is known to cardia...
1,55122,160937.0,Admission Date: [**2192-3-27**] ...,['[(\'Patient underwent uncomplicated penile i...,[(Patient underwent uncomplicated penile impla...
2,88157,168232.0,Admission Date: [**2190-2-23**] ...,"['[(""Patient is a 51 yo female with no cardiac...",[(Patient is a 51 yo female with no cardiac hi...
3,31290,159495.0,Admission Date: [**2201-10-14**] ...,"[""[\n('Patient is 58-year-old man with history...",[(Patient is 58-year-old man with history of D...
4,17341,151110.0,Admission Date: [**2132-6-9**] D...,"[""[\n ('The patient remained on the surgical ...",[(The patient remained on the surgical service...
...,...,...,...,...,...
826,93628,134497.0,Admission Date: [**2177-3-24**] ...,"[""[('Social History: Patient had been maried f...",[(Social History: Patient had been maried for ...
827,14667,172924.0,Admission Date: [**2138-8-7**] D...,"[""[('Pt has >100pk/yrs and continues to smoke'...","[(Pt has >100pk/yrs and continues to smoke, 25..."
828,25235,182008.0,Admission Date: [**2167-6-10**] ...,"['[\n (""52 yo M with PMH DM I, CAD s/p CABG...","[(52 yo M with PMH DM I, CAD s/p CABG x2, asth..."
829,18736,152789.0,Admission Date: [**2109-8-30**] ...,"[""[\n ('Patient recorded as having No Known...",[(Patient recorded as having No Known Allergie...


In [28]:
silver_df_main.iloc[0,-1][1][1]

'427.31'

In [29]:
from typing import Dict, List
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd

class Prepare_Tabulated_Data():
    def __init__(self, code_description: Dict) -> None:
        self.all_labels = list(code_description.keys())[:20]
        self.all_descriptions = list(code_description.values())[:20]
        self.mlb = MultiLabelBinarizer(classes= self.all_labels, sparse_output=False)
    

    
    def fit(self, labels)->None:
        if isinstance(labels, list) and all(isinstance(label, list) for label in labels):
            self.mlb.fit(labels)
        else:
            raise ValueError("Labels should be a list of lists.")
        

    def transform(self, labels)->None:
        if isinstance(labels, list) and all(isinstance(label, list) for label in labels):

            encoded_labels = self.mlb.transform(labels)
            return encoded_labels
        else:
            raise ValueError("Labels should be a list of lists.")   



    def perpare_labels(self, gold_row, silver_row) -> pd.DataFrame:
        gold_encoding = self.transform(labels=[gold_row.ICD9_CODE])


        sentence_x = []
        Silver_vector = []
        Gold_vecor = []
        ICD_Desc = []
        for sentence_and_code in silver_row.doc_sentences:
            
            sentence = sentence_and_code[0]
            silver_code = sentence_and_code[1]

            silver_encoding = self.transform([[silver_code]])

            sentence_x.append(sentence)
            Silver_vector.append(silver_encoding)
            Gold_vecor.append(gold_encoding)
            ICD_Desc.append(self.all_descriptions)
        
        dicti = {'Sentence': sentence_x,
                 'Silver_Labels': Silver_vector,
                 'Gold_Labels': Gold_vecor,
                 'ICD_Desc': ICD_Desc}
        
        df = pd.DataFrame(dicti)

        return df
    
    def Tabulate_Data(self,
                        silver_df: pd.DataFrame,
                        gold_df: pd.DataFrame)-> List:
        
        self.fit([self.all_labels])
        final_ans = []
        for index in silver_df.index:
            silver_row = silver_df.iloc[index]
            gold_row = gold_df.iloc[index]

            label_df = self.perpare_labels(gold_row=gold_row, silver_row=silver_row)
            final_ans.append(label_df)
        
        return final_ans

In [ ]:
encoder = Prepare_Tabulated_Data(code_description=code_descs_dict)
Tabulated_Matrix = encoder.Tabulate_Data(silver_df=silver_df_main, gold_df=gold_df)

In [31]:
Tabulated_Matrix[70]

,Sentence,Silver_Labels,Gold_Labels,ICD_Desc
0,The patient complained of left hip pain.,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,...","[Routine Medical Exam, Screening for Malignant..."
1,Left femoral fracture,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,...","[Routine Medical Exam, Screening for Malignant..."
2,Left femoral fracture extending from the femor...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,...","[Routine Medical Exam, Screening for Malignant..."
3,Left hip pain,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,...","[Routine Medical Exam, Screening for Malignant..."
4,Left femoral fracture extending from the femor...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,...","[Routine Medical Exam, Screening for Malignant..."


## Preparing the For Model

In [32]:
X = []

for matrix in Tabulated_Matrix:
    selected_columns = matrix.iloc[:, [0, -1]]
    X.append(selected_columns)

In [33]:
X[0]

,Sentence,ICD_Desc
0,This 79 year old male who is known to cardiac ...,"[Routine Medical Exam, Screening for Malignant..."
1,He was transfered to [**Hospital1 18**] for fu...,"[Routine Medical Exam, Screening for Malignant..."
2,IMPRESSION: Severe aortic valve stenosis.,"[Routine Medical Exam, Screening for Malignant..."
3,Novolog insulin 8 units [**Hospital1 **] (befo...,"[Routine Medical Exam, Screening for Malignant..."
4,Insulin lispro,"[Routine Medical Exam, Screening for Malignant..."


In [34]:
import torch
from torch.utils.data import DataLoader, Dataset

BATCH_SIZE = 4


# GPU loading
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Prepare Custom Dataset
class Text_Dataset(Dataset):
    def __init__(self, texts):
        self.text = texts
    
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, index):
        return self.text[index]
    

Sentences = [df.iloc[:,0].to_list() for df in Tabulated_Matrix]
ICD_Descriptions = [df.iloc[:,-1].to_list() for df in Tabulated_Matrix]
silver_labels = [df.iloc[:,1].to_list() for df in Tabulated_Matrix]
gold_labels = [df.iloc[:,2].to_list() for df in Tabulated_Matrix]

# Flattening the inputs
Sentences_flattened = [item for sublist in Sentences for item in sublist]
ICD_Descriptions_flattened = [item for sublist in ICD_Descriptions for item in sublist]

# Silver flattened

# Gold flattened


sentence_data = Text_Dataset(Sentences_flattened)
ICD_Descriptions_data = Text_Dataset(ICD_Descriptions_flattened)

sentence_loader = DataLoader(sentence_data, batch_size=BATCH_SIZE, shuffle=False)
ICD_Descriptions_loader = DataLoader(ICD_Descriptions_data, batch_size=BATCH_SIZE, shuffle=False)



## Prepare the Model

In [83]:
from torch import nn
import torch.nn.functional as F
from transformers import DistilBertTokenizer, DistilBertModel

# GPU loading
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Verifier_Model(nn.Module):
    def __init__(self, hidden_units: int = 256, output_shape: int = 1, input_shape: int = 768):
        super().__init__()
        self.tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
        self.TextEncoderModel = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.MLP = nn.Sequential(nn.Linear(in_features=input_shape, out_features=hidden_units),
                                 nn.ReLU(),
                                 nn.Linear(in_features=hidden_units, out_features=output_shape),
                                 )
    
    def TextEncoder(self, text):
        model = self.TextEncoderModel
        tokenizer = self.tokenizer

        model.eval()

        with torch.no_grad():
            inputs = tokenizer(text, padding=True, truncation=True, return_tensors='pt').to(device)
            outputs = model(**inputs)

            latent_rep = outputs.last_hidden_state.mean(dim=1)
        
        return latent_rep
    
    def CalculateSimilarity(self, rep1, rep2):

        rep1 = F.normalize(rep1, p=2, dim=1)
        rep2 = F.normalize(rep2, p=2, dim=1)
        similarity = F.cosine_similarity(rep1, rep2, dim=1)
        return similarity
    
    def forward(self, text1, text2):
        
        rep1 = self.TextEncoder([text1]).to(device)  # Shape: [1, 768]
        rep2 = self.TextEncoder(text2).to(device)    # Shape: [20, 768]

        similarity = self.CalculateSimilarity(rep1, rep2)  # Shape: [1, 20]

        # Element-wise multiplication with similarity scores
        similarity_expanded = similarity.unsqueeze(-1)  # Shape: [1, 20, 1]
        rep1_expanded = rep1.unsqueeze(1)  # Shape: [1, 1, 768]
        weighted_rep1 = rep1_expanded * similarity_expanded  # Shape: [1, 20, 768]

        hf = rep2 + weighted_rep1.squeeze(0)  # Shape: [20, 768]

        # Process each hf vector through the MLP and concatenate the results
        output_list = []
        for i in range(hf.shape[0]):
            output = self.MLP(hf[i].unsqueeze(0))  # Shape: [1, 1]
            output_list.append(output)

        # Stack all outputs and reshape to [1, 20]
        output = torch.cat(output_list, dim=1)  # Shape: [1, 20]

        
        output = torch.sigmoid(output)  # Shape: [1, 20]

        return output


        # return output


    


In [84]:
test_s = Sentences_flattened[0]
text_i = (ICD_Descriptions_flattened[0])

In [85]:
model = Verifier_Model()
model.to(device)
ans = model(test_s, text_i)

In [86]:
ans.shape

torch.Size([1, 20])

In [137]:
input1 = torch.randn(100, 128)
input2 = torch.randn(100, 128)
output = F.cosine_similarity(input1, input2)
print(output.shape)

torch.Size([100])
